In [ ]:


import cv2 as cv
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

img = cv.imread("./shubham.jpg")
# opencv BGR channel format and plt reads images as RGB channel format

img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
plt.imshow(img) # RGB 

from mtcnn.mtcnn import MTCNN

detector = MTCNN()
results = detector.detect_faces(img)

results

x,y,w,h = results[0]['box']

img = cv.rectangle(img, (x,y), (x+w, y+h), (0,0,255), 30)
plt.imshow(img)

my_face = img[y:y+h, x:x+w]
#Facenet takes as input 160x160 
my_face = cv.resize(my_face, (160,160))
plt.imshow(my_face)

my_face

class FACELOADING:
    def __init__(self, directory):
        self.directory = directory
        self.target_size = (160,160)
        self.X = []
        self.Y = []
        self.detector = MTCNN()
    

    def extract_face(self, filename):
        img = cv.imread(filename)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        x,y,w,h = self.detector.detect_faces(img)[0]['box']
        x,y = abs(x), abs(y)
        face = img[y:y+h, x:x+w]
        face_arr = cv.resize(face, self.target_size)
        return face_arr
    

    def load_faces(self, dir):
        FACES = []
        for im_name in os.listdir(dir):
            try:
                path = dir + im_name
                single_face = self.extract_face(path)
                FACES.append(single_face)
            except Exception as e:
                pass
        return FACES

    def load_classes(self):
        for sub_dir in os.listdir(self.directory):
            path = self.directory +'/'+ sub_dir+'/'
            FACES = self.load_faces(path)
            labels = [sub_dir for _ in range(len(FACES))]
            print(f"Loaded successfully: {len(labels)}")
            self.X.extend(FACES)
            self.Y.extend(labels)
        
        return np.asarray(self.X), np.asarray(self.Y)


    def plot_images(self):
        plt.figure(figsize=(18,16))
        for num,image in enumerate(self.X):
            ncols = 3
            nrows = len(self.Y)//ncols + 1
            plt.subplot(nrows,ncols,num+1)
            plt.imshow(image)
            plt.axis('off')

faceloading = FACELOADING("./Original Images")
X, Y = faceloading.load_classes()

plt.figure(figsize=(16,12))
for num,image in enumerate(X):
    ncols = 3
    nrows = len(Y)//ncols + 1
    plt.subplot(nrows,ncols,num+1)
    plt.imshow(image)
    plt.axis('off')

from keras_facenet import FaceNet
embedder = FaceNet()

def get_embedding(face_img):
    face_img = face_img.astype('float32') # 3D(160x160x3)
    face_img = np.expand_dims(face_img, axis=0) 
    # 4D (Nonex160x160x3)
    yhat= embedder.embeddings(face_img)
    return yhat[0] # 512D image (1x1x512)

EMBEDDED_X = []

for img in X:
    EMBEDDED_X.append(get_embedding(img))

EMBEDDED_X = np.asarray(EMBEDDED_X)

np.savez_compressed('faces_embeddings_done_4classes.npz', EMBEDDED_X, Y)

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)

plt.plot(EMBEDDED_X[0]) 
plt.ylabel(Y[0])

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(EMBEDDED_X, Y, shuffle=True, random_state=17)

from sklearn.svm import SVC
model = SVC(kernel='linear', probability=True)
model.fit(X_train, Y_train)

ypreds_train = model.predict(X_train)
ypreds_test = model.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(Y_train, ypreds_train)

accuracy_score(Y_test,ypreds_test)

t_im = cv.imread("shubham.jpg")
t_im = cv.cvtColor(t_im, cv.COLOR_BGR2RGB)
x,y,w,h = detector.detect_faces(t_im)[0]['box']

t_im = t_im[y:y+h, x:x+w]
t_im = cv.resize(t_im, (160,160))
test_im = get_embedding(t_im)

test_im = [test_im]
ypreds = model.predict(test_im)

ypreds

encoder.inverse_transform(ypreds)

print(encoder.classes_)  # This will print all recognized names


import pickle
#save the model
with open('svm_model_160x160.pkl','wb') as f:
    pickle.dump(model,f)

import cv2 as cv
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import pickle
from keras_facenet import FaceNet

# Suppress TensorFlow Warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Initialize FaceNet Model
facenet = FaceNet()

# Load Face Embeddings and Labels
faces_embeddings = np.load("faces_embeddings_done_4classes.npz")
Y = faces_embeddings['arr_1']

# Encode Labels
encoder = LabelEncoder()
encoder.fit(Y)

# Load Face Detection Model
haarcascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")

# Load Pre-Trained SVM Model
try:
    model = pickle.load(open("svm_model_160x160.pkl", 'rb'))
except FileNotFoundError:
    print("Error: SVM model file not found!")
    exit()

# Define Confidence Threshold
THRESHOLD = 30  # Adjust as needed

# Open Webcam
cap = cv.VideoCapture(0)  # Use 0 for default webcam
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Real-Time Face Recognition Loop
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    rgb_img = cv.cvtColor(frame, cv.COLOR_BGR2RGB)  # Convert to RGB
    gray_img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)  # Convert to Grayscale

    # Detect Faces
    faces = haarcascade.detectMultiScale(gray_img, scaleFactor=1.2, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        face_img = rgb_img[y:y+h, x:x+w]
        face_img = cv.resize(face_img, (160, 160))
        face_img = np.expand_dims(face_img, axis=0)

        try:
            # Get Face Embeddings
            ypred = facenet.embeddings(face_img)
            
            # Predict Face Label
            probabilities = model.predict_proba(ypred)
            confidence = np.max(probabilities) * 100
            
            if confidence >= THRESHOLD:
                face_name = model.predict(ypred)
                final_name = encoder.inverse_transform(face_name)[0]
            else:
                final_name = "Unknown"

            # Draw Bounding Box & Label
            cv.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 255), 3)
            cv.putText(frame, f"{final_name} ({confidence:.2f}%)", (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 
                       1, (0, 0, 255), 2, cv.LINE_AA)
        except Exception as e:
            print(f"Prediction Error: {e}")

    # Display Output
    cv.imshow("Face Recognition", frame)

    # Press 'q' to Exit
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Release Resources
cap.release()
cv.destroyAllWindows()





































